In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import pandas_datareader as pdr

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 백테스트 날짜, 초기 잔고, 리밸런싱 주기
start = datetime(2020, 8, 8)
end = datetime(2021, 1, 9)
init_balance = 1000

#rebalancing_months = [1,2,3,4,5,6,7,8,9,10,11,12] # 리밸런싱 주기 : 한달로 고정
rebalancing_target = -1 # -1 월 말, 0 월 초
rebalancing_dates = [0, 1, 2] # 월말 3영업일, 월초 2영업일

fee = 0.5 # 거래비용 및 슬리피지. %(퍼센트) 단위

# 모멘텀 측정 기간
mtum_period = [1, 3, 6] #월단위
mtum_ratio = [1, 1, 1]


portfolio = []
# 포트폴리오 1
portfolio.append({
        'asset': ['143850.KS', '232080.KS'],
        'ratio': [1, 1],
        'name': '개인연금'
})
# 포트폴리오 2
portfolio.append({
#         'asset': ['SPY', 'SCZ'],
        'asset': ['360750.KS', '232080.KS'],
        'ratio': [1, 1],
        'name': '원본'
})
# Exit 자산
portfolio.append({
        'asset': ['304660.KS'],
#         'asset': ['TLT'],
        'ratio': [1],
        'name': 'Exit Asset'
})
# 벤치마크: S&P500
portfolio.append({
        'asset': ['^GSPC'], # S&P500
#         'asset': ['143850.KS'], # S&P500
        'ratio': [1],
        'name': 'Benchmark'
})

In [3]:
portfolio

[{'asset': ['143850.KS', '232080.KS'], 'ratio': [1, 1], 'name': '개인연금'},
 {'asset': ['360750.KS', '232080.KS'], 'ratio': [1, 1], 'name': '원본'},
 {'asset': ['304660.KS'], 'ratio': [1], 'name': 'Exit Asset'},
 {'asset': ['^GSPC'], 'ratio': [1], 'name': 'Benchmark'}]

# 주가 데이터 가져오기

In [4]:
def get_price(port): # 개별 포트폴리오의 가격 리스트에대한 df 생성

    # 각 종목 별 수정 주가 데이터를 크롤링 해 리스트로 시계열 데이터 시리즈를 값으로 갖는 리스트 생성
    price_list = []
    columns_name = []
    for i in range(len(port)):
        for asset in port[i]['asset']:
            # 포트 별 가격 데이터 가져오기
            price_list.append(pdr.get_data_yahoo(asset, start, end)['Adj Close'])
#             price_list.append(pdr.DataReader(asset, 'google', start, end)['Adj Close'])
            # 컬럼 명 생성
            if i < len(port)-2:
                columns_name.append(f'Port{i+1}_'+f'{asset}')
            else:
                columns_name.append(asset)

    df_list = pd.concat(price_list, axis=1) #주가 데이터 리스트를 데이터프레임으로 변환
    df_list.columns = columns_name # 포트를 구성하는 종목 명을 컬럼 명으로
    first_day = df_list.dropna().index.min() # 전 종목 결측치 없는 최초 일자를 거래시작일로 지정
    df_price = df_list[first_day:].fillna(method='pad') # 최초일 이후의 결측치는 전일종가로 채움
    end_day = df_price.index.max()

    return df_price

In [5]:
df_price = get_price(portfolio)
df_price

NotImplementedError: data_source='google' is not implemented

# 리밸런싱 날짜 계산

In [ ]:
# 리밸런싱 날짜 계산하기
df_price['Date'] = df_price.index
#df_price['month'] = df_price.index.month
df_price['year_month'] = df_price.index.strftime('%Y-%m')

#df_target = df_price.loc[df_price["month"].isin(rebalancing_months)].groupby('year_month').apply(lambda x: x.iloc[rebalancing_target])
df_target = df_price.groupby('year_month').apply(lambda x: x.iloc[rebalancing_target])
#rebal_list = list(df_target['Date'])

df_price = df_price.drop(['year_month', 'Date'], axis=1)

# 모멘텀 스코어 계산

In [ ]:
asset_list = df_price.columns[:-2]
asset_list

In [ ]:
# 시기별 모멘텀 df 만들기
for asset in asset_list:
    df_price[f'{asset}_score'] = 0
    for i, rebal_day in zip(range(len(df_target["Date"])), df_target["Date"]):
        mtum_score = 0
        for mtum_p, mtum_r  in zip(mtum_period, mtum_period):
            if i <= mtum_p-1:
                tmp_mtum = (df_price[asset].loc[rebal_day]/df_price[asset].loc[df_target["Date"].iloc[0]] - 1) * (mtum_r/sum(mtum_ratio))
            else:
                mtum_day = df_target["Date"].iloc[i-mtum_p]
                tmp_mtum = (df_price[asset].loc[rebal_day]/df_price[asset].loc[mtum_day] - 1) * (mtum_r/sum(mtum_ratio))
            mtum_score += tmp_mtum
        df_price[f'{asset}_score'].loc[rebal_day] = mtum_score

In [ ]:
df_price.loc[df_target["Date"]]

# 모멘텀 스코어 비교

In [ ]:
for j in range(len(portfolio)-2):
    asset1 = portfolio[j]['asset'][0]
    asset2 = portfolio[j]['asset'][1]
    for i, rebal_day in zip(range(len(df_target["Date"])), df_target["Date"]):
        if (df_price.loc[ rebal_day, f'Port{j+1}_{asset1}_score' ] > 0) and (df_price.loc[ rebal_day, f'Port{j+1}_{asset1}_score' ] > df_price.loc[ rebal_day, f'Port{j+1}_{asset2}_score' ]):
            hold_asset = f'Port{j+1}_{asset1}'
        elif (df_price.loc[ rebal_day, f'Port{j+1}_{asset2}_score' ] > 0) and (df_price.loc[ rebal_day, f'Port{j+1}_{asset2}_score' ] > df_price.loc[ rebal_day, f'Port{j+1}_{asset1}_score' ]):
            hold_asset = f'Port{j+1}_{asset2}'
        else:
            hold_asset = portfolio[-2]['asset'][0]
        df_price.loc[ rebal_day, f'Port{j+1}_hold'] = hold_asset

# 포트폴리오 별 잔액 계산

In [ ]:
for j in range(len(portfolio)-2):
    df_price.loc[df_target["Date"][0], f'Port{j+1}_bal'] = init_balance
    for i in range(len(df_target["Date"])-1):
        hold = df_price.loc[df_target["Date"][i], f'Port{j+1}_hold']
        df_price.loc[df_target["Date"][i]:df_target["Date"][i+1], f'Port{j+1}_bal'] = df_price.loc[df_target["Date"][i], f'Port{j+1}_bal'] * df_price.loc[df_target["Date"][i]:df_target["Date"][i+1], hold] / df_price.loc[df_target["Date"][i], hold]

In [ ]:
Benchmark = portfolio[-1]['asset'][0]
df_price.loc[df_target["Date"][0]:, f'{Benchmark}_bal'] = init_balance * df_price.loc[df_target["Date"][0]:, Benchmark] / df_price.loc[df_target["Date"][0], Benchmark]

In [ ]:
port_bal = df_price.loc[df_target["Date"][0]:,df_price.columns[-(len(portfolio)-1):]]
# port_bal.columns = port_bal.columns.str.replace('_bal','')

In [ ]:
port_bal

In [ ]:
# port_list = [portfolio[i]['name'] for i in range(len(portfolio)-2)]
# port_list.append(Benchmark)
# port_list

In [ ]:
def performance_indicator(etf): # etf는 df["etf명"]
    date = etf.index.tolist()
    balance = etf.values.tolist()
    ###########최종 평가액, 첫거래일, 마지막거래일, 테스트기간###########
    final_balance = balance[-1]
    first_day, last_day = date[0], date[-1]
    time_period = ((last_day - first_day).days)/365
    
    CAGR = ((balance[-1]/balance[0]) ** (1 / float(time_period)) - 1) * 100
    cum_return = ((balance[-1]/balance[0] - 1) * 100)
    ###########신고가, MDD, 일 변동률###############################
    historic_high = 0
    historic_high_percent = 0
    MDD = 0
    drawdown_list = [0]
    daily_change_list = [0]
    
    for i in range(1, len(date)):     
        if historic_high < balance[i]:
            historic_high = balance[i] 

        drawdown_list.append((balance[i]/historic_high - 1) * 100)
        
        if MDD > drawdown_list[i]:
            MDD = drawdown_list[i]
        
        daily_change_list.append((balance[i]/balance[i-1] - 1)*100)
        
    historic_high_percent = (historic_high/balance[0] - 1)*100
    ###########STDEV, 샤프지수신고가################################
    # 일단위 변동성 연율화 하기 위해 252일 영업일 기준으로 루트 252 곱 함
    STDEV = np.std(daily_change_list) * np.sqrt(252) 
    sharpe_ratio = np.mean(daily_change_list) / np.std(daily_change_list) * np.sqrt(252) 
    
    my_str = []
    result_list = [init_balance, final_balance, CAGR, MDD, STDEV, sharpe_ratio]
    for my_number in result_list:
        if my_number in [CAGR, MDD]:
            my_str.append(str(round(my_number, 2))+' %')
        elif my_number in [final_balance]:
            my_str.append(str(round(my_number)))
        else : 
            my_str.append(str(round(my_number, 2)))
    
    first, last = first_day.strftime('%Y-%m-%d'), last_day.strftime('%Y-%m-%d')
    print(f'테스트기간 : {first} ~ {last} / {round(time_period,1)} 년')
    
    return my_str, drawdown_list

In [ ]:
tmp = {
        'index': [],
        'data': [],
        'drawdown': []
    }
########### 포트폴리오 리턴 결과 표 ###############################
# port_list = [portfolio[i]['name'] for i in range(len(portfolio))]
port_list = port_bal.columns
for port in port_list:
    tmp['index'].append(port)
    tmp['data'].append(performance_indicator(port_bal[port])[0])
    tmp['drawdown'].append(performance_indicator(port_bal[port])[1])

portfolio_return = pd.DataFrame(data=tmp['data'], index=tmp['index'], \
                   columns=['Init Balance', 'Final Balance', 'CAGR', 'MDD', 'STDEV', 'Sharpe Ratio'])

# ########### drawdown 리스트 -> 데이터프레임 변환 ###################
tmp_drawdown = [pd.DataFrame(data=tmp['drawdown'][i], index=port_bal.index, columns=[port_list[i]]) for i in range(len(tmp['drawdown']))]
df_drawdown = pd.concat(tmp_drawdown, axis=1)

########### Annual Returns / Monthly Returns ###################
port_bal['year'] = port_bal.index.strftime('%Y')
port_bal['year_month'] = port_bal.index.strftime('%Y-%m')
annual = port_bal.groupby('year').apply(lambda x: x.iloc[[-1]]).reset_index('Date').drop(['year_month', 'year', 'Date'], axis=1)
monthly = port_bal.groupby('year_month').apply(lambda x: x.iloc[[-1]]).reset_index('Date').drop(['year_month', 'year', 'Date'], axis=1)

for port in port_list:
    for i in (range(len(annual)-1, 0, -1)):
        annual[port][i] = round((annual[port][i] / annual[port][i-1] - 1), 4) * 100
    for i in (range(len(monthly)-1, 0, -1)):
        monthly[port][i] = round((monthly[port][i] / monthly[port][i-1] - 1), 4) * 100
annual.iloc[0] = (annual.iloc[0] / init_balance - 1) * 100
monthly.iloc[0] = (monthly.iloc[0] / init_balance - 1) * 100

In [ ]:
portfolio_return

In [ ]:
# 그래픽 크기 설정
plt.rcParams["font.size"] = 16
plt.rcParams["figure.figsize"] = (20, 10)

port_bal.plot()
df_drawdown.plot()
annual.plot.bar()
monthly.plot.bar()
plt.grid()
plt.legend()
plt.show()